## MSBX 5420 Assignment 3
This assignment is about Spark Machine Learning and Spark Streaming. First two tasks focus on machine learning, and the third one combines machine learning and streaming analysis. We will use IMDB reviews data for the whole assignment.

### Task 1 - Topic Modeling on Moive Reviews with Spark ML
First of all, let's load the data. The data structure is very simple.One column is review text, and another column is the label of review sentiment (positive or negative). Same as exercise, we can load .csv.gz file directly from spark.

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[4]').config("spark.executor.memory", "1g").config("spark.driver.memory", "2g").appName('spark_ml_imdb').getOrCreate()
#for cluster - change kernel to PySpark
#spark = SparkSession.builder.master('spark://spark-master:7077').appName('spark_ml_imdb').getOrCreate()

In [ ]:
reviews = spark.read.options(inferSchema = True, multiLine = True, escape = '\"').csv('IMDB_Reviews.csv.gz', header=True)
#for cluster
#reviews = spark.read.options(inferSchema = True, multiLine = True, escape = '\"').csv('s3://msbx5420-spr21/zhiyiwang/IMDB_Reviews.csv.gz', header=True)
reviews.show()

First, we should clean up the review texts. Besides those special characters we have tried to remove in exercise, here we also need to remove the html tags in the text.

In [ ]:
import pyspark.sql.functions as fn
import pyspark.ml.feature as ft

#remove html tags in the text with regular expression
reviews = reviews.withColumn('review', fn.regexp_replace(fn.col("review"), '<[^>]+>', ' '))
#remove special characters and line breaks in the text with regular expression
reviews = reviews.withColumn('review', fn.regexp_replace(fn.col("review"), '([^\s\w_]|_)+', ' ')).withColumn('review', fn.regexp_replace(fn.col("review"), '[\n\r]', ' '))
reviews.take(1)

Now let's create tokenizer to start the data processing.

In [ ]:
tokenizer = ft.RegexTokenizer(inputCol='review', outputCol='review_tok', pattern='\s+|[,.\"/!]')
tokenizer.transform(reviews).select('review_tok').take(1)

Then remove stopwords in the text.

In [ ]:
stopwords = ft.StopWordsRemover(inputCol=tokenizer.getOutputCol(), outputCol='review_stop')
stopwords.transform(tokenizer.transform(reviews)).select('review_stop').take(1)

Now same as what we did in the exercise, let's create `CountVectorizer` to transform the text into term frequency vector. 

In [ ]:
tf = ft.CountVectorizer(inputCol=stopwords.getOutputCol(), outputCol='review_tf')
tokenized = stopwords.transform(tokenizer.transform(reviews))
tf.fit(tokenized).transform(tokenized).select('review_tf').take(1)

Then we use the `LDA` model to do topic modeling. We create the model here with 30 topics.

In [ ]:
import pyspark.ml.clustering as clus
lda = clus.LDA(k=30, optimizer='online', maxIter=10, featuresCol=tf.getOutputCol())

Now let's build the pipeline to train the topic model from the raw data. It will take a while to run.

In [ ]:
from pyspark.ml import Pipeline

#[Your Code] to build a ML pipeline to fit LDA


#topics = pipeline_model.transform(reviews)
#topics.select('topicDistribution').take(5)

Let's see if we have properly discovered the topics. This is just the same code we display topics in the exercise - we will reuse it several times here.

In [ ]:
#Code to extract topics from models
vectorized_model = pipeline_model.stages[2]
topic_model = pipeline_model.stages[3]
vocab = vectorized_model.vocabulary
topic_words_list = topic_model.describeTopics(20)
topic_words_rdd = topic_words_list.rdd
topics_words = topic_words_rdd.map(lambda row: row['termIndices']).map(lambda idx_list: [vocab[idx] for idx in idx_list]).collect()

for idx, topic in enumerate(topics_words):
    print("topic: {}".format(idx))
    print("*"*25)
    for word in topic:
       print(word)
    print("*"*25)

How do you think about the topics? Do they make sense? If you think the topics we get from the movie reviews should be better, let's continue to see what we can do to make them better.

One possible reason is that we have many words that do not show up frequently. That is, they are very specific words to certain movies but don't occur across reviews. Such words are not very meaningful and they do not represent common themes in those reviews. So here we limit the frequency of words to at least 5 and run LDA with pipeline again.

In [ ]:
#filter the countvectorizer
tf = ft.CountVectorizer(inputCol=stopwords.getOutputCol(), outputCol='review_tf', minDF=5)

In [ ]:
#[Your Code] to build a ML pipeline and fit LDA again


#topics = pipeline_model.transform(reviews)
#topics.select('topicDistribution').take(5)

In [ ]:
#Code to extract topics from models
vectorized_model = pipeline_model.stages[2]
topic_model = pipeline_model.stages[3]
vocab = vectorized_model.vocabulary
topic_words_list = topic_model.describeTopics(20)
topic_words_rdd = topic_words_list.rdd
topics_words = topic_words_rdd.map(lambda row: row['termIndices']).map(lambda idx_list: [vocab[idx] for idx in idx_list]).collect()

for idx, topic in enumerate(topics_words):
    print("topic: {}".format(idx))
    print("*"*25)
    for word in topic:
       print(word)
    print("*"*25)

It is expected that the topics are getting better but still not very satisfying. There are lots of words shown in different topics many times; possibly they are too common so they shouldn't be that important. Let's take one more step to use TF-IDF vector rather than TF vector. To build IF-IDF, we first create TF with CountVectorizer then create IDF from TF vector. Then we run LDA model with IF-IDF vector.

In [ ]:
#use tf-idf vector
tf = ft.CountVectorizer(inputCol=stopwords.getOutputCol(), outputCol="review_tf", vocabSize=10000)
idf = ft.IDF(inputCol=tf.getOutputCol(), outputCol="review_tfidf", minDocFreq=5)

In [ ]:
#[Your Code] to create a LDA model (30 topics) and put everything together into a ML pipeline to fit LDA


#topics = pipeline_model.transform(reviews)
#topics.select('topicDistribution').take(5)

In [ ]:
#Code to extract topics from models
tf_model = pipeline_model.stages[2]
topic_model = pipeline_model.stages[4]
vocab = tf_model.vocabulary
topic_words_list = topic_model.describeTopics(20)
topic_words_rdd = topic_words_list.rdd
topics_words = topic_words_rdd.map(lambda row: row['termIndices']).map(lambda idx_list: [vocab[idx] for idx in idx_list]).collect()

for idx, topic in enumerate(topics_words):
    print("topic: {}".format(idx))
    print("*"*25)
    for word in topic:
       print(word)
    print("*"*25)

The topics should be more reasonable now. You should believe they can still be further improved by cleaning up the text and tuning the hyperparameters but let's stop here for assignment. If you want to try yourself beyond the assignment, you can change the model configuration to see if you can get any further improvement.

### Task 2 - Movie Review Sentiment Analysis with Spark ML
The second task we are going to prediction. Let's continue with the reviews data and now we can do sentiment analysis with the TF-IDF. So with the TF-IDF vector, we can train and predict review sentiment.

In [ ]:
#first let's confirm the potential labels
#(it is possible sentiment can be neutral so we should make sure if that's the case)
reviews.select('sentiment').distinct().show()

In [ ]:
#let's create the binary numerical lable from postive/negative
reviews = reviews.withColumn('sentiment_label', fn.when(fn.col('sentiment')=='positive', 1.0).otherwise(0.0))
reviews.show()

In [ ]:
#split the training and testing set, with 80/20
reviews_train, reviews_test = reviews.randomSplit([0.8, 0.2], seed=200)

In [ ]:
import pyspark.ml.classification as cl
from pyspark.ml import Pipeline

#craete logistic gression model and then build the pipeline to train the model
lr = cl.LogisticRegression(maxIter=10, labelCol='sentiment_label', featuresCol=idf.getOutputCol())

In [ ]:
#[Your Code] to build a ML pipeline and train logistic regression model



In [ ]:
#make predictions with pipeline model
predictions = lr_model.transform(reviews_test)

In [ ]:
import pyspark.ml.evaluation as ev
#model evaluation for binary classification
evaluator = ev.BinaryClassificationEvaluator(rawPredictionCol='probability', labelCol='sentiment_label')
print(evaluator.evaluate(predictions, {evaluator.metricName: 'areaUnderROC'}))
print(evaluator.evaluate(predictions, {evaluator.metricName: 'areaUnderPR'}))

The prediction performance looks acceptable. Here note that TF-IDF is a long vector (here we select top 10000 words, but still a large number), so let's try something different. As mentioned in the class, another way to model text is word embedding with the Word2Vec model. So next we create word vector and use it to predict sentiment.

In [ ]:
#create word2vec model
word2vec = ft.Word2Vec(vectorSize=100, minCount=5, inputCol=stopwords.getOutputCol(), outputCol="review_word2vec")

In [ ]:
#same logistic regression model, but take output from word2vec model
#[Your Code] to create a logistic regression model and build pipeline with word2vec to train logistic regession; then make predictions and evaluate model (areaUnderROC and areaUnderPR)



Check the prediction performance with only 100 features; word2vec model is a very useful representation of words and it reduces dimensionality significantly. 

In the end, let's try an alternative model. In classfication, Support Vector Machine (SVM) is commonly used and let's see how we can use it here. We will keep the orginal configuration of word2vec model (vector size is 100) here for SVM.

In [ ]:
#same word2vec model configuration is adopted here
word2vec = ft.Word2Vec(vectorSize=100, minCount=5, inputCol=stopwords.getOutputCol(), outputCol="review_word2vec")
#create svm with LinearSVC, with features from word2vec model outputCol
svm = cl.LinearSVC(maxIter=10, labelCol='sentiment_label', featuresCol=word2vec.getOutputCol())

In [ ]:
#build the ml pipeline and train the model; then make predictions 
#[Your Code] to build the ML pipeline to train SVM model; then make predictions (no need to evaluate, the evaluation is slightly different here so provided below)



In [ ]:
#model evaluation, here slightly different for LinearSVC
evaluator = ev.BinaryClassificationEvaluator(rawPredictionCol='rawPrediction', labelCol='sentiment_label')
print(evaluator.evaluate(predictions, {evaluator.metricName: 'areaUnderROC'}))
print(evaluator.evaluate(predictions, {evaluator.metricName: 'areaUnderPR'}))

### Task 3 - Combine Spark ML and Streaming Analysis
Now we have our sentiment prediction model with acceptable predictive performance. The last task is to combine this machine learning mode with spark streaming. That is, with a data stream, we will use the trained model to make real-time predictions. We will still use IMDB reviews and here we will create a simulated data stream from files in a folder, then receive the review stream and predict sentiment using spark structured streaming.

We will first create multiple files so that we can simulate a data stream to read files incrementally from a folder. To do that, we use the code below - it creates a folder 'review_stream', turn the spark dataframe into pandas dataframe (this is not a recommended approach but here we use it for convenience and conciseness), split the data into 100 smaller ones, and then save each smaller csv file into the folder.

In [ ]:
reviews_test_stream = reviews_test.withColumn('review_id', fn.monotonically_increasing_id())

In [ ]:
#if you use cluster, do not run the code below, just stream from s3 directory with code for cluster in the next cell 

import pandas as pd
import numpy as np
import os

#code for local mode
#here we just create a local directory to simulate the data stream
if not os.path.exists('review_stream'):
    os.mkdir('review_stream')

#here we use pandas at local docker to speed up, with spark it is slow
df = reviews_test_stream.orderBy('review_id').toPandas()
#here we write a small number of rows into each csv file and read data stream from these files
#we can split the dataframe into 100 smaller ones so each one has 100 rows
for i,chunk in enumerate(np.array_split(df, 100)):
    chunk.to_csv('./review_stream/reviews_{}.csv'.format(i), index=False)

Now we can read from the stream. For convenience, we just take the existing spark dataframe to reuse the schema.

In [ ]:
#read from data stream in a folder
streaming_review = spark.readStream.schema(reviews_test_stream.schema).option("maxFilesPerTrigger", 1).csv('./review_stream')
#for cluster
#streaming_review = spark.readStream.schema(reviews_test_stream.schema).option("maxFilesPerTrigger", 1).csv('s3://msbx5420-spr21/zhiyiwang/review_stream')

Here from the data stream, we want to know two results. First, how many positive or negative reviews we have received in real time? Second, how many positive or negative reviews in each time window (so we know whether there is a peak of positive or negative review in certain time)? So we will do some calculations, and to capture time window, we will use the current timestamp to create 'processing_time' (the time we receive the data) and apply window on this timestamp.

In [ ]:
streaming_review_time = streaming_review.withColumn('processing_time', fn.current_timestamp())

In [ ]:
#we can just take the pipeline model we have trained to make prediction
#[Your Code] to use the last pipeline model with SVM to make predictions on 'streaming_review_time'; save the result as 'streaming_sentiment'



In [ ]:
#create a predicted text label from 'predicted column' is the prediction
streaming_sentiment = streaming_sentiment.withColumn('predicted', fn.when(fn.col('prediction')==1.0, 'positive').otherwise('negative'))

#here we do transformations to get the results we need
#first, get total number of positive and negative reviews we have received
streaming_sentiment_count = streaming_sentiment.groupBy('predicted').count()

#second, still number of positive and negative reviews we received, but by time window (60 seconds)
streaming_sentiment_window_count = streaming_sentiment.groupBy(fn.window('processing_time', '60 seconds'), 'predicted').count()

In [ ]:
#now we have two streaming dataframe results
print(streaming_sentiment_count.isStreaming)
print(streaming_sentiment_window_count.isStreaming)

In [ ]:
#now we can define query to start streaming analysis and set the result table as 'sentiment'
query_sentiment = (streaming_sentiment_count.writeStream.format("memory").queryName("sentiment").outputMode("complete").start())

In [ ]:
#define another query for the result table of windowed positive and negative review counts
#[Your Code] to define the second query, name of result table is 'sentiment_window'



In [ ]:
#query the first result table to monitor real time results
spark.sql('select * from sentiment').show()

In [ ]:
#query the second result table to monitor real time results, order the results by window
spark.sql('select * from sentiment_window order by window').show(truncate=False)

In [ ]:
#stop query to finish streaming analysis
query_sentiment.stop()

In [ ]:
#stop query to finish streaming analysis
query_sentiment_window.stop()